In [1]:
from __future__ import division
from __future__ import print_function

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import random
import config
import torch.optim as optim
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# NEURAL NETWORK MODULES/LAYERS
from dataset import Dataset
# METRICS CLASS FOR EVALUATION
from metrics import Metrics
# CONFIG PARSER
from config import parse_args
# TRAIN AND TEST HELPER FUNCTIONS
from trainer import Trainer
import glob
from tqdm import tqdm
from PIL import Image
import torchvision
from torchvision import models
# from tensorboardX import SummaryWriter

import model_vgg16
# import model_vgg19
# import model_resnet50
# import model_densenet121
# import model_inceptionv3

# MAIN BLOCK
# def main():
# global args
# args = parse_args()
# global logger


In [2]:
torch.__version__

'1.2.0'

In [3]:
class config():
    def __init__(self):
        self.data='./data'
        self.save='checkpoints/'
        self.expname='raw_vgg16_holistic_multiple_loss'
        self.expno=0
        self.pretrained_model='vgg16'
        self.num_classes=16
        # training arguments
        self.epochs=100000
        self.batchsize=50
        self.lr=0.0001
        self.wd=1e-4
        self.sparse=False
        self.optim='adam'
        # miscellaneous options
        self.seed=123
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.pretrained_holistic=0

#         cuda_parser = parser.add_mutually_exclusive_group(required=False)
#         cuda', dest='cuda', action='store_true')
#         cuda_parser.add_argument('--no-cuda', dest='cuda', action='store_false')
#         parser.set_defaults(cuda=True)
        

In [4]:
args = config()
# writer = SummaryWriter(comment = args.expname)
args.batchsize

50

In [5]:

torch.manual_seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

if not os.path.exists(args.save):
    os.makedirs(args.save)



In [6]:
def create_optimizer(args, parameters):
    if args.optim=='adam':
        optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='adagrad':
        optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='sgd':
        optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim == 'adadelta':
        optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)
    return optimizer

In [7]:
# initialize model, criterion/loss_function, optimizer
pretrained_vgg16 = models.vgg16(pretrained=True)

# Freeze training for all layers
# for child in pretrained_vgg16.children():
#     for param in child.parameters():
#         param.requires_grad = False
start_point = 0
if args.pretrained_holistic == 0 and args.expname == 'vgg_holistic_375_500':
    model = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
    optimizer = create_optimizer(args, list(model.parameters()))
    
    if os.path.exists('./checkpoints/vgg_holistic_375_500.pt'):
        checkpoint = torch.load('./checkpoints/vgg_holistic_375_500.pt')
        model.load_state_dict(checkpoint['model'])
        if args.cuda:
            model.cuda()
        else:
            start_point = 0
        optimizer.load_state_dict(checkpoint['optim'])

elif args.pretrained_holistic == 0 and args.expname != 'vgg_holistic_375_500':
    model = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
    optimizer = create_optimizer(args, list(model.parameters()))
    if os.path.exists('./checkpoints/{0}.pt'.format(args.expname)):
        checkpoint = torch.load('./checkpoints/{0}.pt'.format(args.expname))
        model.load_state_dict(checkpoint['model'])

elif args.pretrained_holistic == 1:
    checkpoint_holistic = torch.load('./checkpoints/vgg_holistic_375_500.pt')
    vgg16 = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
    pretrained_holistic_model = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
    pretrained_holistic_model.load_state_dict(checkpoint_holistic['model'])

    model = model_vgg16.DocClassificationRest(args, vgg16, pretrained_holistic_model)

    parameters = list(pretrained_holistic_model.parameters()) #+ list(model.parameters()) + list(vgg16.parameters())

    if os.path.exists('./checkpoints/{0}.pt'.format(args.expname)):
        checkpoint_ensemble = torch.load('./checkpoints/{0}.pt'.format(args.expname))
        model.load_state_dict(checkpoint_ensemble['model'])
        if args.cuda:
            model.cuda()

        start_point = checkpoint_ensemble['epoch']
        optimizer = create_optimizer(args, parameters)
        optimizer.load_state_dict(checkpoint_ensemble['optim'])
    else:
        optimizer = create_optimizer(args, parameters)
        #optimizer.load_state_dict(checkpoint_holistic['optim'])    

criterion = nn.CrossEntropyLoss(reduction='mean')


if args.cuda:
    model.cuda(), criterion.cuda()


metrics = Metrics(args.num_classes)

# create trainer object for training and testing
trainer = Trainer(args, model, criterion, optimizer)

In [8]:
print(model)
for parameter, name in zip(model.parameters(), model.named_parameters()):
     print("Param Name: " + name[0] + " Requires Training: " + str(parameter.requires_grad))
print(optimizer)

DocClassificationHolistic(
  (pretrained_model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): Re

In [9]:
# train_dir = glob.glob(os.path.join(args.data,'train/holistic/*.jpg'))
# dev_dir = glob.glob(os.path.join(args.data,'val/holistic/*.jpg'))
# test_dir = glob.glob(os.path.join(args.data,'test/holistic/*.jpg'))

# print(len(train_dir), len(dev_dir), len(test_dir))

# 'data/labels/train.csv'
# train_dataset = Dataset('data/stratified/rvl-cdip/labels/train.csv',model_type=args.pretrained_model)
# dev_dataset = Dataset('data/stratified/rvl-cdip/labels/val.csv', model_type=args.pretrained_model)
test_dataset = Dataset('data/stratified/rvl-cdip/labels/test.csv', model_type=args.pretrained_model)


# print('==> Size of train data   : %d ' % len(train_dataset))
# print('==> Size of val data   : %d ' % len(dev_dataset))
print('==> Size of test data   : %d ' % len(test_dataset))

# train_idx = list(np.arange(len(train_dataset)))
# dev_idx = list(np.arange(len(dev_dataset)))
test_idx = list(np.arange(len(test_dataset)))

best = float('inf')
columns = ['ExpName','ExpNo', 'Epoch', 'Loss','Accuracy']
results = []
early_stop_count = 0

==> Size of test data   : 15999 


In [10]:
# bh,h,f,l,r,lab = train_dataset[np.random.randint(55000,60000,64)]
# batch_holistic = torch.unsqueeze(bh , dim=1).repeat(1,3,1,1).float().cuda()
# # bh = np.array(bh.detach().cpu())
# print(bh.shape, batch_holistic.shape)
# batch_header = torch.unsqueeze(h, dim=1).repeat(1,3,1,1).float().cuda()
# batch_footer = torch.unsqueeze(f, dim=1).repeat(1,3,1,1).float().cuda()
# batch_left = torch.unsqueeze(l, dim=1).repeat(1,3,1,1).float().cuda()
# batch_right = torch.unsqueeze(r, dim=1).repeat(1,3,1,1).float().cuda()

# all_data = [batch_holistic,batch_header,batch_footer,batch_left,batch_right]
# writer.add_graph(model, input_to_model=all_data)
# # writer.flush()


In [11]:
# from matplotlib import pyplot as plt
# def plot_data(x,ax):
# #     x = x.reshape((size,size))
    
#     ax.imshow(x, cmap='gray')
# #     if y is not None:
# #         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
# def plot_images(x,batch_size=64):
#     fig = plt.figure(figsize=(50,50))
#     fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.01, wspace=0.01)
#     for i in range(batch_size):
#         ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
#     #     print(y['classes'][i])
#     #     print(np.argmax(c[i]))
#         plot_data(x[i], ax)
# plot_images(bh)
# print(l)
# # display(plot_images(np.squeeze(seg[0],axis=-1)))

In [12]:
batch_test_idxs = []
counter = 0

while counter<len(test_dataset):
    batch_test_idx = np.array(list(np.arange(counter, counter+args.batchsize)))
    batch_test_idxs.append(batch_test_idx)
    counter+=args.batchsize
batch_test_idxs

test_acc = []
test_predictions = []
test_labels = []


# ----------------- Test ---------------------
for batch_test_idx in tqdm(batch_test_idxs, desc='Test batches..'):
#     print(batch_test_idx)
    try:
        test_batch_holistic, \
                test_batch_header, \
                test_batch_footer, \
                test_batch_left_body, \
                test_batch_right_body, \
                test_batch_labels = test_dataset[batch_test_idx]

        if args.pretrained_holistic == 0 and 'multiple_loss' not in args.expname: # singleloss
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_holistic(test_batch_holistic, test_batch_labels)
        elif args.pretrained_holistic == 0 and 'multiple_loss' in args.expname: # multipleloss
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_rest_multiple_loss(test_batch_holistic, \
                                                                                    test_batch_header, \
                                                                                    test_batch_footer, \
                                                                                    test_batch_left_body, \
                                                                                    test_batch_right_body, \
                                                                                    test_batch_labels)
        elif args.pretrained_holistic == 1:
            test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_rest(test_batch_holistic, \
                                    test_batch_header, \
                                    test_batch_footer, \
                                    test_batch_left_body, \
                                    test_batch_right_body, \
                                    test_batch_labels)
    #         test_predictions.append(test_batch_predictions)
    #         test_labels.append(test_batch_labels)
    #         test_loss = test_loss + test_batch_loss

        test_accuracy = metrics.accuracy(test_batch_predictions, test_batch_labels)
    #     print(test_accuracy)
        test_acc.append(test_accuracy)
    except Exception as ex:
        print(ex)
        



Test batches..:   0%|          | 0/320 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
'Accuracy of {0}: {1} %'.format(args.expname, np.round(np.mean(test_acc),2)*100)


In [ ]:
test_acc